In [137]:
import random
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
import pyzx
from typing import List,Tuple
from pyzx.graph.base import BaseGraph, VT, ET

In [ ]:
class SimulatedAnnealer:
    """
    Attributes:
        initial_temp (float): 初期温度
        final_temp (float): 最終温度
        cooling_rate (float): 冷却率 (0 < alpha < 1)
        max_iterations (int): 各温度での最大試行回数
    """

    def __init__(self, initial_temp, final_temp, cooling_rate, max_iterations):
        self.initial_temp = initial_temp
        self.final_temp = final_temp
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations

    def _acceptance_probability(self, o, new_score, temp):
        """
        悪い解への遷移を受理する確率を計算する。
        exp(-delta_E / T)
        """
        if new_score < old_score:
            return 1.0
        # new_score >= old_score の場合（スコアが悪化した場合）
        delta_score = new_score - old_score
        return math.exp(-delta_score / temp)

    def solve(self, initial_state, evaluate_func, get_neighbor_func):
        """
        焼きなまし法を実行して最適解を探す。

        Args:
            initial_state: 初期状態
            evaluate_func: 状態を評価する関数
            get_neighbor_func: 近傍の状態を生成する関数

        Returns:
            (best_state, best_score): 見つかった最も良い状態とその評価値
        """
        current_temp = self.initial_temp
        current_state = copy.deepcopy(initial_state)
        current_score = evaluate_func(current_state)

        best_state = current_state
        best_score = current_score

        history = {'temp': [], 'score': []}

        while current_temp > self.final_temp:
            for _ in range(self.max_iterations):
                # 1. 近傍の状態を生成
                neighbor_state = get_neighbor_func(current_state)

                # 2. 新しい状態を評価
                neighbor_score = evaluate_func(neighbor_state)

                # 3. 遷移を許容するか決定
                prob = self._acceptance_probability(current_score, neighbor_score, current_temp)
                if random.random() < prob:
                    current_state = neighbor_state
                    current_score = neighbor_score

                # 4. 最良解を更新
                if current_score < best_score:
                    best_state = current_state
                    best_score = current_score
            
            # 履歴を保存
            history['temp'].append(current_temp)
            history['score'].append(best_score)

            # 5. 温度を更新（冷却）
            current_temp *= self.cooling_rate

        return best_state, best_score, history
    
    """
    #状態のスコアを計算
    def score():

    def get_neighbor_func():
    

    #グラフに対して、簡約化操作とそのノード一覧
    def get_action():
    
        Args:
        initial_state: 初期状態
        evaluate_func: 状態を評価する関数
        get_neighbor_func: 近傍の状態を生成する関数

    Returns:
        (best_state, best_score): 見つかった最も良い状態とその評価値

"""


In [139]:
def generate_H_S_CNOT_T_circuit(qubits, gates, p_t=0, seed=1000):
    random.seed(seed)  
    p_s = 0.333 * (1.0 - p_t)  
    p_had = 0.333 * (1.0 - p_t)  
    p_cnot = 0.333 * (1.0 - p_t)  

    c = pyzx.Circuit(qubits) 
    for _ in range(gates):
        r = random.random() 
        if r < p_had:
            c.add_gate("HAD", random.randrange(qubits))
        elif r < p_had + p_s:
            c.add_gate("S", random.randrange(qubits))
        elif r < p_had + p_s + p_t:
            c.add_gate("T", random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl != tgt:
                    break
            c.add_gate("CNOT", tgt, ctrl)
    return c


In [ ]:
#actions
def get_actions(g:BaseGraph):
    a = pyzx.rules.match_lcomp_parallel(g)
    b = pyzx.rules.match_pivot_parallel(g)

    return a,b

def apply_lcomp(g:BaseGraph ,match: Tuple[int,List[int]]):
    print(match)
    etab, rem_verts, rem_edges, check_isolated_vertices = pyzx.rules.lcomp(g,[match])
    g.add_edge_table(etab)
    g.remove_edges(rem_edges)
    g.remove_vertices(rem_verts)
    if check_isolated_vertices: g.remove_isolated_vertices()
    return g

def apply_pivot(g:BaseGraph, match: Tuple[int,int ,List[int],List[int]]):
    print(match)
    etab, rem_verts, rem_edges, check_isolated_vertices = pyzx.rules.pivot(g, [match])
    g.add_edge_table(etab)
    g.remove_edges(rem_edges)
    g.remove_vertices(rem_verts)
    if check_isolated_vertices: g.remove_isolated_vertices()
    return g

def print_graph_data(g: BaseGraph):
    boundarycount= 0
    for v in g.vertices():

        if (g.type(v) == 0):
            vtype = "Boundary"
            boundarycount +=1
        else:
            vtype = "Z"

        phase = g.phase(v)
        neighbors = g.neighbors(v)
        print(f"Vertex {v}:")
        print(f"  Type: {vtype}")
        print(f"  Phase: {phase}")
        print(f"  Neighbors: {neighbors}")
    
    for e in g.edges():
        s, t = g.edge_st(e)
        etype = g.edge_type(e)
        print(f"Edge {e}: {s} --({etype})-- {t}")
    print("boudarycount",boundarycount)

def get_gate_num(g:BaseGraph):
    g_tmp = g.copy()
    c = pyzx.extract.extract_circuit(g_tmp,up_to_perm=True)
    c = pyzx.optimize.basic_optimization(c)
    a = c.stats_dict()
    dict = {}
    dict["all"] = a["gates"]
    dict["two"] = a["twoqubit"]
    dict["one"] = a["gates"] - a["twoqubit"]
    dict["t"] = a["tcount"]
    return dict

def get_node_and_edge_num(g:BaseGraph) -> Tuple[int, int]:
    a= g.num_vertices()
    b = g.num_edges()
    return a,b

def score(g:BaseGraph) -> int:
    a = get_gate_num(g)
    score = 10 * a["two"] + a["one"]

    return score
    

def get_neighbors_state_and_score(g: BaseGraph):
    can_lcomp, can_pivot = get_actions(g)
    neighbors_state = []
    neighbors_score = []

    for i in range(len(can_lcomp)):
        g_tmp = g.copy()
        apply_lcomp(g_tmp, can_lcomp[i])  # 状態を書き換え
        neighbors_state.append(g_tmp)
        neighbors_score.append(score(g_tmp))


    for i in range(len(can_pivot)):
        g_tmp = g.copy()
        apply_pivot(g_tmp, can_pivot[i])  # 状態を書き換え
        neighbors_state.append(g_tmp)
        neighbors_score.append(score(g_tmp))

    return neighbors_state, neighbors_score


In [ ]:
c = generate_H_S_CNOT_T_circuit(4, 100, p_t=0.08, seed=1000)
g = c.to_graph()
pyzx.draw(g)
pyzx.simplify.spider_simp(g)
pyzx.simplify.to_gh(g) #red node -> green
pyzx.draw(g)
pyzx.simplify.id_simp(g)
pyzx.draw(g)
print_graph_data(g)


spider_simp: 25. 13. 11. 4. 3. 1.  6 iterations


id_simp: 18. 2. 1.  3 iterations


Vertex 0:
  Type: Boundary
  Phase: 0
  Neighbors: dict_keys([5])
Vertex 1:
  Type: Boundary
  Phase: 0
  Neighbors: dict_keys([10])
Vertex 2:
  Type: Boundary
  Phase: 0
  Neighbors: dict_keys([11])
Vertex 3:
  Type: Boundary
  Phase: 0
  Neighbors: dict_keys([4])
Vertex 4:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([3, 5, 6])
Vertex 5:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([0, 4, 12])
Vertex 6:
  Type: Z
  Phase: 1/2
  Neighbors: dict_keys([4, 7])
Vertex 7:
  Type: Z
  Phase: 3/4
  Neighbors: dict_keys([6, 39])
Vertex 10:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([11, 13, 1])
Vertex 11:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([2, 10, 21])
Vertex 12:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([5, 28, 21])
Vertex 13:
  Type: Z
  Phase: 3/4
  Neighbors: dict_keys([10, 35])
Vertex 21:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([12, 11, 27])
Vertex 27:
  Type: Z
  Phase: 0
  Neighbors: dict_keys([28, 29, 21])
Vertex 28:
  Type: Z
  Phase: 3/4
  Neighbors: dict_keys([27, 12,

In [103]:
pyzx.draw(g)

v, neighbors = pyzx.rules.match_lcomp_parallel(g)[0]
etab, rem_verts, rem_edges, check_isolated_vertices = pyzx.rules.lcomp(g, [(v, neighbors)])
g.add_edge_table(etab)
g.remove_edges(rem_edges)
g.remove_vertices(rem_verts)
if check_isolated_vertices: g.remove_isolated_vertices()

pyzx.draw(g)


IndexError: list index out of range